In [1]:
from os import path, environ
import pandas as pd
import numpy as np

data_dir = '/home/ehramovich/netology/IV/data/'
mrec_dir = '/home/ehramovich/netology/IV/mrec/sbin'


columns = ['usersha1','artmbid','artname', 'plays']
data = pd.read_csv(path.join(data_dir, 'lastfm_small.tsv'), sep='\t', header=None, names=columns)
data.head()

,usersha1,artmbid,artname,plays
0,00000c289a1829a808ac09c00daf10bc3c4e223b,3bd73256-3905-4f3a-97e2-8b341527f805,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,f2fb0ff0-5679-42ec-a55c-15109ce6e320,die Ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,b3ae82c2-e60b-4551-a76d-6620f1b456aa,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,3d6bbeb7-f90e-4d10-b440-e153c0d10b53,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,bbd2ffd7-17f4-4506-8572-c1ea58c3f9a8,juliette & the licks,706


In [2]:
data.fillna('None', inplace=True)

#+1 for futher model quality estimation by MREC
data['user_id'] = data['usersha1'].astype("category").cat.codes.copy() + 1
data['artist_id'] = data['artmbid'].astype("category").cat.codes.copy() + 1
data.drop(['usersha1', 'artmbid', 'artname'], axis=1, inplace=True)
data.head()

,plays,user_id,artist_id
0,2137,1,15531
1,1099,1,63469
2,897,1,46858
3,717,1,15968
4,706,1,48969


In [3]:
data.describe()

,plays,user_id,artist_id
count,1000000.000000,1000000.000000,1000000.000000
mean,216.606950,10232.925996,33678.492236
std,604.378024,5912.022447,19230.330182
min,1.000000,1.000000,1.000000
25%,34.000000,5118.000000,17298.000000
50%,94.000000,10237.000000,34544.000000
75%,225.000000,15347.000000,49488.000000
max,135392.000000,20465.000000,66799.000000


In [4]:
%%time
test_indices = np.random.choice(
    data.index.values,
    replace=False,
    size=int(len(data.index.values) * 0.2)
)
test_data = data.iloc[test_indices]
train_data = data.drop(test_indices)

CPU times: user 110 ms, sys: 12.6 ms, total: 123 ms
Wall time: 151 ms


In [5]:
test_user_set = set(test_data["user_id"].unique())
train_user_set = set(train_data["user_id"].unique())
print("нет в обучающей выборке, но есть в тестовой: {}".format(
    len(test_user_set - train_user_set)))
print("нет в тестовой выборке, но есть в обучающей: {}".format(
    len(train_user_set - test_user_set)))
print("всего пользователей: {}".format(len(data["user_id"].unique())))

нет в обучающей выборке, но есть в тестовой: 1
нет в тестовой выборке, но есть в обучающей: 24
всего пользователей: 20465


In [6]:
# исключим таких пользователей из тестовой и обучающей выборок
user_ids_to_exclude = (test_user_set - train_user_set).union(train_user_set - test_user_set)
bad_indices = test_data[test_data["user_id"].isin(user_ids_to_exclude).values].index
test_data.drop(bad_indices, inplace=True)
bad_indices = train_data[train_data["user_id"].isin(user_ids_to_exclude).values]
train_data.drop(bad_indices.index, inplace=True)

/home/ehramovich/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [7]:
test_file_name = "lastfm.test.0"
test_data[["user_id", "artist_id", "plays"]].to_csv(
    path.join(data_dir, test_file_name),
    sep="\t",
    header=False,
    index=False
)
train_file_name = "lastfm.train.0"
train_data[["user_id", "artist_id", "plays"]].to_csv(
    path.join(data_dir, train_file_name),
    sep="\t",
    header=False,
    index=False
)

In [8]:
# функция, которая красиво печатает информацию о разреженных матрицах
from scipy.sparse import csr_matrix

def sparse_info(sparse_matrix: csr_matrix) -> None:
    print("Размерности матрицы: {}".format(sparse_matrix.shape))
    print("Ненулевых элементов в матрице: {}".format(sparse_matrix.nnz))
    print("Доля ненулевых элементов: {}"
          .format(sparse_matrix.nnz / sparse_matrix.shape[0] / sparse_matrix.shape[1])
    )
    print("Среднее значение ненулевых элементов: {}".format(sparse_matrix.data.mean()))
    print("Максимальное значение ненулевых элементов: {}".format(sparse_matrix.data.max()))
    print("Минимальное значение ненулевых элементов: {}".format(sparse_matrix.data.min()))

In [18]:
# создаём разреженную матрицу item*user
from scipy.sparse import coo_matrix
import numpy as np

plays = coo_matrix((
    train_data["plays"].astype(np.float64),
    (
        train_data["artist_id"],
        train_data["user_id"]
    )
))

sparse_info(plays.tocsr())

Размерности матрицы: (66799, 20466)
Ненулевых элементов в матрице: 799800
Доля ненулевых элементов: 0.0005850304471920511
Среднее значение ненулевых элементов: 216.5659902475619
Максимальное значение ненулевых элементов: 134993.0
Минимальное значение ненулевых элементов: 1.0


# NN Cosine

In [25]:
import time
from implicit.nearest_neighbours import CosineRecommender

nn_model = CosineRecommender()
print("строим матрицу схожести по косинусной мере")
start = time.time()
nn_model.fit(plays)
print("построили матрицу схожести по косинусной мере за {} секунд".format(
        time.time() - start))
sparse_info(nn_model.similarity)

строим матрицу схожести по косинусной мере
построили матрицу схожести по косинусной мере за 0.22775983810424805 секунд
Размерности матрицы: (66799, 66799)
Ненулевых элементов в матрице: 1215728
Доля ненулевых элементов: 0.0002724560758406711
Среднее значение ненулевых элементов: 0.4436296188097192
Максимальное значение ненулевых элементов: 1.9999999999999996
Минимальное значение ненулевых элементов: 0.0


In [26]:
print("получаем рекомендации для всех пользователей")
start = time.time()
user_plays = plays.T.tocsr()
with open(path.join(data_dir, "recs", test_file_name + ".recs.tsv"), "w") as output_file:
    for user_id in test_data["user_id"].unique():
        for artist_id, score in nn_model.recommend(user_id, user_plays):
                output_file.write("%s\t%s\t%s\n" % (user_id, artist_id, score))
print("получили рекомендации для всех пользователей за {} секунд".format(
        time.time() - start))

получаем рекомендации для всех пользователей
получили рекомендации для всех пользователей за 10.714582204818726 секунд


In [27]:
environ["DATA_DIR"] = data_dir
environ["MREC_DIR"] = mrec_dir

# используем mrec в качестве метрики качества
!$MREC_DIR/mrec_evaluate \
    --input_format=tsv --test_input_format=tsv \
    --train $DATA_DIR/lastfm.test.0 \
    --recsdir $DATA_DIR/recs

[2017-12-14 16:50:21,433] INFO: processing /home/ehramovich/netology/IV/data/lastfm.test.0...
None
mrr            0.0486 +/- 0.0000
prec@5         0.0169 +/- 0.0000
prec@10        0.0146 +/- 0.0000
prec@15        0.0097 +/- 0.0000
prec@20        0.0073 +/- 0.0000


# ALS

In [22]:
from implicit.als import AlternatingLeastSquares
environ['OPENBLAS_NUM_THREADS'] = '1'

als_model = AlternatingLeastSquares()
als_model.fit(plays)

In [23]:
print("получаем рекомендации для всех пользователей")
start = time.time()
user_plays = plays.T.tocsr()
with open(path.join(data_dir, "recs-als", test_file_name + ".recs.tsv"), "w") as output_file:
    for user_id in test_data["user_id"].unique():
        for artist_id, score in als_model.recommend(user_id, user_plays):
                output_file.write("%s\t%s\t%s\n" % (user_id, artist_id, score))
print("получили рекомендации для всех пользователей за {} секунд".format(
        time.time() - start))

получаем рекомендации для всех пользователей
получили рекомендации для всех пользователей за 75.90379858016968 секунд


In [24]:
# используем mrec в качестве метрики качества
!$MREC_DIR/mrec_evaluate \
    --input_format=tsv --test_input_format=tsv \
    --train $DATA_DIR/lastfm.test.0 \
    --recsdir $DATA_DIR/recs-als

[2017-12-14 16:49:14,997] INFO: processing /home/ehramovich/netology/IV/data/lastfm.test.0...
None
mrr            0.3146 +/- 0.0000
prec@5         0.1379 +/- 0.0000
prec@10        0.1126 +/- 0.0000
prec@15        0.0751 +/- 0.0000
prec@20        0.0563 +/- 0.0000
